In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [6]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [8]:
import evaluate

accuracy = evaluate.load("accuracy")

# to calculate accurecy

In [9]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [10]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

## now train

In [11]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 3
batches_per_epoch = len(tokenized_imdb["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [13]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [14]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_imdb["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_imdb["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [15]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

for some accurecy bullshit


In [16]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

now train


In [17]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3
1562/1562 [==============================] - 1896s 1s/step - loss: 0.2496 - val_loss: 0.1885
Epoch 2/3
1562/1562 [==============================] - 1867s 1s/step - loss: 0.1282 - val_loss: 0.2071
Epoch 3/3
1562/1562 [==============================] - 1826s 1s/step - loss: 0.0622 - val_loss: 0.2147


In [68]:
!mkdir model
model.save("./model/")

model.save_pretrained("./model/pretrained")
tokenizer.save_pretrained("./model/pretrained")

mkdir: cannot create directory ‘model’: File exists


('./model/pretrained/tokenizer_config.json',
 './model/pretrained/special_tokens_map.json',
 './model/pretrained/vocab.txt',
 './model/pretrained/added_tokens.json',
 './model/pretrained/tokenizer.json')

In [69]:
!ls ./model/pretrained

from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./model/pretrained")
model = TFAutoModelForSequenceClassification.from_pretrained("./model/pretrained")

config.json		 tf_model.h5		tokenizer.json
special_tokens_map.json  tokenizer_config.json	vocab.txt


Some layers from the model checkpoint at ./model/pretrained were not used when initializing TFDistilBertForSequenceClassification: ['dropout_158']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./model/pretrained and are newly initialized: ['dropout_178']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [80]:
text = "This was a worst movie. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the not to watch movies."
inputs = tokenizer(text, return_tensors="tf")

In [81]:
logits = model(**inputs).logits

In [82]:
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

'NEGATIVE'

In [95]:
!du -h ./model/pretrained
!ls -ls ./model/pretrained

!zip -r pretrained.zip ./model/pretrained
from google.colab import files
files.download('pretrained.zip')


257M	./model/pretrained
total 262616
     4 -rw-r--r-- 1 root root       653 Aug 10 20:26 config.json
     4 -rw-r--r-- 1 root root       125 Aug 10 20:27 special_tokens_map.json
261680 -rw-r--r-- 1 root root 267955144 Aug 10 20:27 tf_model.h5
     4 -rw-r--r-- 1 root root       320 Aug 10 20:27 tokenizer_config.json
   696 -rw-r--r-- 1 root root    711494 Aug 10 20:27 tokenizer.json
   228 -rw-r--r-- 1 root root    231508 Aug 10 20:27 vocab.txt
  adding: model/pretrained/ (stored 0%)
  adding: model/pretrained/tokenizer_config.json (deflated 43%)
  adding: model/pretrained/vocab.txt (deflated 53%)
  adding: model/pretrained/config.json (deflated 44%)
  adding: model/pretrained/special_tokens_map.json (deflated 42%)
  adding: model/pretrained/tf_model.h5 (deflated 8%)
  adding: model/pretrained/tokenizer.json (deflated 71%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [102]:
!du -ah


4.0K	./.config/active_config
4.0K	./.config/.last_update_check.json
4.0K	./.config/logs/2023.08.09/13.36.50.146065.log
4.0K	./.config/logs/2023.08.09/13.35.48.465148.log
12K	./.config/logs/2023.08.09/13.36.14.412731.log
40K	./.config/logs/2023.08.09/13.35.21.813012.log
4.0K	./.config/logs/2023.08.09/13.36.50.985175.log
4.0K	./.config/logs/2023.08.09/13.36.23.243759.log
72K	./.config/logs/2023.08.09
76K	./.config/logs
4.0K	./.config/.last_survey_prompt.yaml
12K	./.config/default_configs.db
0	./.config/config_sentinel
4.0K	./.config/gce
4.0K	./.config/configurations/config_default
8.0K	./.config/configurations
4.0K	./.config/.last_opt_in_prompt.yaml
120K	./.config
236M	./pretrained.zip
4.0K	./config.json
256M	./tf_model.h5
4.0M	./model/saved_model.pb
4.0K	./model/fingerprint.pb
4.0K	./model/pretrained/tokenizer_config.json
228K	./model/pretrained/vocab.txt
4.0K	./model/pretrained/config.json
4.0K	./model/pretrained/special_tokens_map.json
256M	./model/pretrained/tf_model.h5
696K	./model/